In [ ]:
from hstrat import hstrat
from iterpop import iterpop as ip
from keyname import keyname as kn
from matplotlib import pyplot as plt
from natsort import natsorted
from nbmetalog import nbmetalog as nbm
import pandas as pd
import seaborn as sns
import sympy
from teeplot import teeplot as tp


In [ ]:
nbm.print_metadata()


# How many strata are retained under the recency-proportional resolution policy?


In [ ]:
records = [{
    'Generations Elapsed': generations_elapsed,
    'Resolution': str(resolution),
    'Num Strata Retained': hstrat\
        .StratumRetentionPredicateRecencyProportionalResolution(resolution)\
        .CalcNumStrataRetainedExact(generations_elapsed),
}
    for generations_elapsed in (10**3, 10**6, 10**9, 10**12)
    for resolution in (1, 2, 4, 10, 100)
]
df = pd.DataFrame.from_records(records)
df


In [ ]:
piv = pd.pivot(
    df,
    values=['Num Strata Retained'],
    index=['Generations Elapsed'],
    columns=['Resolution'],
)
# piv.columns = piv.columns.map(', Resolution '.join)
piv[piv.index.name] = piv.index
piv.reset_index(drop=True, inplace=True)
piv = piv.reindex(natsorted(piv.columns), axis=1)
# piv.columns.names = (None, None)
piv


In [ ]:
piv.to_csv(kn.pack({
    'a': 'space-complexity',
    'policy': 'recency-proportional-resolution',
    'ext': '.csv',
}))


In [ ]:
def loglineplot(*args, **kwargs):
    sns.lineplot(*args, **kwargs)
    plt.gca().set_xscale('log')

tp.tee(
    loglineplot,
    data=df[df['Resolution'].astype(int) < 100],
    x='Generations Elapsed',
    y='Num Strata Retained',
    hue='Resolution',
    marker='o',
    teeplot_outattrs={
        'policy': 'recency-proportional-resolution',
    },
)


# How does recency-proportional stratum retention scale with resolution?


From <https://github.com/mmore500/hstrat/blob/3eebabb6931b3ef3bbf6b7808a6e136caf600bd8/hstrat/hstrat/stratum_retention_predicates/StratumRetentionPredicateRecencyProportionalResolution.py#L222> we have space complexity of the recency-proportional resolution as less than or equal to

$\begin{align*}
\log(x) + \sum_{i=1}^r \log\Big(\frac{n}{i}\Big)
\end{align*}$

where $r$ is resolution parameter and $n$ is number of strata deposited.
(This is within a constant factor, glossing over $\log$ vs. $\log_2$, omitting integer truncated division, and substituting $\log(x)$ for integer bit weight.)

Applying logarithm properties,

$\begin{align*}
\sum_{i=1}^r \log\Big(\frac{n}{i}\Big)
&= \log\Big(\prod_{i=1}^r \frac{n}{i}\Big)\\
&= \log\Big(\frac{\prod_{i=1}^r n}{\prod_{i=1}^r i}\Big)\\
&= \log\Big(\frac{n^r}{r!}\Big).
\end{align*}$

So, we can re-express space complexity as

$\begin{align*}
\log(n) + \log\Big(\frac{n^r}{r!}\Big).
\end{align*}$

In order to understand how space complexity scales with respect to resolution $r$, we can consider the ratio of space complexity between a policy with resolution $r$ and a policy with twice the resolution $2r$ as $r$ goes to infinity.
Let's apply computer algebra to evaluate this limit.


In [ ]:
# specify variables with assumptions for domain
r = sympy.Symbol('r', integer=True, positive=True, real=True,)
n = sympy.Symbol('n', integer=True, positive=True, real=True,)

space_complexity = sympy.log(n**r / sympy.factorial(r)) + sympy.log(n)
space_complexity


In [ ]:
ratio = space_complexity.subs(r, 2*r) / space_complexity
ratio


Evaluating as $n \to \infty$,


In [ ]:
lim = sympy.Limit(ratio, n, sympy.oo)
lim


In [ ]:
lim.doit().simplify()


Evaluating as $r \to \infty$,


In [ ]:
lim = sympy.Limit(ratio, r, sympy.oo)
lim


In [ ]:
lim.doit()


## Result

Doubling resolution doubles space complexity.
So, in the limit, space complexity scales linearly with desired resolution.
